In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

### Scraping the Premier League match results for the 2020-21, 2021-22, 2022-23, 2023-24, and 2024-25 seasons

In [5]:
base_url = "https://fixturedownload.com/view/json/epl-"

In [41]:
all_matches = []
# looping through the websites for the premier league match results for the 20-21, 21-22, 22-23, 23-24, and 24-25 seasons
for year in range(2020,2025):
    year_url = base_url + str(year)
    year_website = requests.get(year_url)        
    soup = BeautifulSoup(year_website.content, 'html.parser')
    data = soup.find('textarea')
    data_text = data.get_text()
    matches = json.loads(data_text)
    season = str(year) + '-' + str(year+1)
    for match in matches:
        match['Season'] = season
    all_matches.append(matches)

In [42]:
# Right now all_matches is a list of lists of dictionaries. This converts it to a lists of dictionaries.
all_matches_collapsed = [m for season in all_matches for m in season]

### Putting the results in a dataframe

In [43]:
raw_matches_df = pd.DataFrame(all_matches_collapsed)

In [44]:
raw_matches_df.head()

,MatchNumber,RoundNumber,DateUtc,Location,HomeTeam,AwayTeam,Group,HomeTeamScore,AwayTeamScore,Season
0,4,1,2020-09-12 11:30:00Z,Craven Cottage,Fulham,Arsenal,None,0,3,2020-2021
1,3,1,2020-09-12 14:00:00Z,Selhurst Park,Crystal Palace,Southampton,None,1,0,2020-2021
2,5,1,2020-09-12 16:30:00Z,Anfield,Liverpool,Leeds,None,4,3,2020-2021
3,8,1,2020-09-12 19:00:00Z,London Stadium,West Ham,Newcastle,None,0,2,2020-2021
4,7,1,2020-09-13 13:00:00Z,The Hawthorns,West Brom,Leicester,None,0,3,2020-2021


In [45]:
len(raw_matches_df)

1900

In [46]:
print(1900/5)

380.0


We see that there are 1900 total matches in the dataframe. We have the data for 5 seasons, so this is what we expect because there is 380 matches per season (38 weeks and 10 matches per week).